# Azure Kusto
In this notebook we will access Azure Kusto database

In [18]:
# install all the azure kusto modules
%pip install azure.kusto.data
%pip install azure-kusto-ingest
%pip install azure.identity
%pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder, ClientRequestProperties
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd
import json

class KustoConnection():
    def __init__(self, configuration : dict):
        uri = configuration.get('cluster_uri')
        if (not str(uri).startswith('https://')):
            uri = "https://" + uri
        if (not str(uri).endswith('.kusto.windows.net')) :
            uri = uri + '.kusto.windows.net'
        self.cluster_uri = uri
        self.database = configuration.get('database')
        cred = ChainedTokenCredential(ManagedIdentityCredential(), AzureCliCredential(), DefaultAzureCredential())
        connection_string = KustoConnectionStringBuilder.with_aad_application_token_authentication(self.cluster_uri, 
                                                                                                   cred.get_token(self.cluster_uri + '/.default').token)        
        # Create a KustoClient instance
        self.client = KustoClient(connection_string)

    def query(self, query: str, properties: ClientRequestProperties = None) -> pd.DataFrame:
        """
        Execute a Kusto query.
        @param query: The query.
        @returns: A Pandas DataFrame.
        """
        response = self.client.execute_query(self.database, query, properties)
        df = dataframe_from_result_table(response.primary_results[0])
        return df

    def get_schema(self, table_name: str) -> dict:
        """
        Get a table schema
        @param table_name: name of table
        @returns a dictionary mapping column names to kusto types
        """
        schema = self.client.execute_mgmt(self.database, f".show table {table_name} schema as json").primary_results[0][0]['Schema']
        return json.loads(schema)

    def close(self):
        """
        Close the Kusto client.
        """
        self.client.close()

class DCShortCodeSelection():
    def __init__(self):
        self.kusto_client = KustoConnection({'cluster_uri' : "dpxedwhprod.southcentralus", 'database': "Dpxedwhprod"});
        self.dcshortcode_selection = {
        'T2_DC_List' : 
        '''
        let IFP_Done = DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        |summarize by ProjectName, Tranche
        | join kind = leftouter  
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, 
              ProjectName,Tranche, 
              L2Milestone = "Design IFP", 
              ProjectType, 
              IFP_TaskStartDate = DesignIFP, 
              IFP_TaskFinishDate = DesignIFP
        ) 
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2
        | where IFP_TaskFinishDate <= ago(1d);
        let CR_NotDone = DCEPAllMilestonesCurrent 
        | where Tranche == 2 
        | summarize by DCShortCode, Tranche, 
                      L2Milestone = "Colo Ready", 
                      ProjectType, 
                      CR_TaskStartDate = ColoReadyDate, 
                      CR_TaskFinishDate = ColoReadyDate 
        | where CR_TaskFinishDate > ago(2d); 
        IFP_Done 
        | join kind=inner(CR_NotDone) 
        on $left.ProjectType == $right.ProjectType and $left.DCShortCode == $right.DCShortCode 
        | summarize by DCShortCode 
        | order by DCShortCode asc
        ''',
        
        'T2_IST2CR_List' : 
        '''
        let IFPFullSet = (DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche
        | join kind = leftouter  
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, 
            L2Milestone = "Design IFP", 
            ProjectType, IFP_TaskStartDate = DesignIFP, 
            IFP_TaskFinishDate = DesignIFP
        ) on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2); 
        let CSFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,
                Tranche, L2Milestone = "Construction Start", 
                ProjectType, CS_TaskStartDate = ConstructionDate, 
                CS_TaskFinishDate = ConstructionDate
            ) on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
            | where Tranche ==2); 
            let FSFullSet = 
            ( 
                EstimatedFoundationStart | where Tranche ==2 
                | summarize by DCShortCode, ProjectName,
                Tranche, L2Milestone="Foundation", ProjectType, 
                FS_TaskStartDate = Foudation_TaskStartDate, 
                FS_TaskFinishDate = Foudation_TaskFinishDate
            ); 
            let ISTFullSet = 
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, Tranche, 
                L2Milestone = "IST Complete", ProjectType, 
                IST_TaskStartDate = ISTDate, IST_TaskFinishDate = ISTDate
            ); 
            let CRFullSet =  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, Tranche, 
                  L2Milestone = "Colo Ready", ProjectType, 
                  CR_TaskStartDate = ColoReadyDate, 
                  CR_TaskFinishDate = ColoReadyDate
            ); 
            IFPFullSet 
            | join kind= inner (CSFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
            | join kind= inner (FSFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
            | join kind= inner (ISTFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType
            | join kind= inner (CRFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
            | where IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  <= ago(1d) 
               and FS_TaskStartDate <= ago(1d) and IST_TaskStartDate <= ago(1d) 
               and CR_TaskStartDate > ago(1d) | summarize by DCShortCode
        ''',
        
        'T2_FS2CR_List' : 
        '''
        let IFPFullSet = 
        (
            DCEPAllMilestonesCurrent 
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  (DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, 
              L2Milestone = "Design IFP", ProjectType, IFP_TaskStartDate = DesignIFP, 
              IFP_TaskFinishDate = DesignIFP
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2); 
        let CSFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,Tranche, 
                  L2Milestone = "Construction Start", ProjectType, 
                  CS_TaskStartDate = ConstructionDate, CS_TaskFinishDate = ConstructionDate
            )
            on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
            | where Tranche ==2); 
            let FSFullSet = 
            ( 
                EstimatedFoundationStart
                | where Tranche ==2
                | summarize by DCShortCode, ProjectName,Tranche, 
                  L2Milestone="Foundation", ProjectType, 
                  FS_TaskStartDate = Foudation_TaskStartDate, 
                  FS_TaskFinishDate = Foudation_TaskFinishDate
            ); 
            let ISTFullSet = 
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, Tranche, L2Milestone = "IST Complete", 
                  ProjectType, IST_TaskStartDate = ISTDate, IST_TaskFinishDate = ISTDate
            ); 
            let CRFullSet = 
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, Tranche, L2Milestone = "Colo Ready", 
                   ProjectType, CR_TaskStartDate = ColoReadyDate, 
                   CR_TaskFinishDate = ColoReadyDate
            ); 
            IFPFullSet | join kind= inner (CSFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
            | join kind= inner (FSFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
            | join kind= inner (ISTFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType
            | join kind= inner (CRFullSet) 
            on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
            | where IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  <= ago(1d) 
               and FS_TaskStartDate <= ago(1d) and IST_TaskStartDate > ago(1d) 
               and CR_TaskStartDate > ago(1d) 
            | summarize by DCShortCode
        ''',
        
        'T2_CS2CR_List' :
        ''' 
        let IFPFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,Tranche, L2Milestone = "Design IFP", 
                  ProjectType, IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
            ) on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1 
            | where Tranche ==2
        ); 
        let CSFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,Tranche, 
                  L2Milestone = "Construction Start", ProjectType, 
                  CS_TaskStartDate = ConstructionDate, CS_TaskFinishDate = ConstructionDate
            ) on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
            | where Tranche ==2
        ); 
        let FSFullSet = 
        ( 
            EstimatedFoundationStart
            | where Tranche ==2
            | summarize by DCShortCode, ProjectName,Tranche, L2Milestone="Foundation", 
              ProjectType, FS_TaskStartDate = Foudation_TaskStartDate, 
              FS_TaskFinishDate = Foudation_TaskFinishDate
        ); 
        let ISTFullSet = 
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, Tranche, L2Milestone = "IST Complete", 
              ProjectType, IST_TaskStartDate = ISTDate, IST_TaskFinishDate = ISTDate
        ); 
        let CRFullSet =  
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, Tranche, L2Milestone = "Colo Ready", 
              ProjectType, CR_TaskStartDate = ColoReadyDate, CR_TaskFinishDate = ColoReadyDate
        ); 
        IFPFullSet 
        | join kind= inner (CSFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | join kind= inner (FSFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | join kind= inner (ISTFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType
        | join kind= inner (CRFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | where IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  <= ago(1d) 
          and FS_TaskStartDate  > ago(1d) and IST_TaskStartDate > ago(1d) 
          and CR_TaskStartDate > ago(1d) 
        | summarize by DCShortCode
        ''',
        
        'T2_IFP2CR_List' :
        '''
        let IFPFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche| join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,Tranche, L2Milestone = "Design IFP", 
                  ProjectType, IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
            )
            on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
            | where Tranche ==2
        ); 
        let CSFullSet = 
        (
            DCEPAllMilestonesCurrent
            | where ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by ProjectName, Tranche
            | join kind = leftouter  
            (
                DCEPAllMilestonesCurrent 
                | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
                | summarize by DCShortCode, ProjectName,Tranche, 
                  L2Milestone = "Construction Start", ProjectType, 
                  CS_TaskStartDate = ConstructionDate, CS_TaskFinishDate = ConstructionDate
            )
            on $left.ProjectName == $right.ProjectName 
            | project-away ProjectName1, Tranche1
            | where Tranche ==2
        ); 
        let FSFullSet = 
        ( 
            EstimatedFoundationStart
            | where Tranche ==2
            | summarize by DCShortCode, ProjectName,Tranche, 
              L2Milestone="Foundation", ProjectType, 
              FS_TaskStartDate = Foudation_TaskStartDate, 
              FS_TaskFinishDate = Foudation_TaskFinishDate
        ); 
        let ISTFullSet = 
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, Tranche, L2Milestone = "IST Complete", 
              ProjectType, IST_TaskStartDate = ISTDate, IST_TaskFinishDate = ISTDate
        ); 
        let CRFullSet=  
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 2 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, Tranche, L2Milestone = "Colo Ready", 
              ProjectType, CR_TaskStartDate = ColoReadyDate, CR_TaskFinishDate = ColoReadyDate
        ); 
        IFPFullSet 
        | join kind= inner (CSFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | join kind= inner (FSFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | join kind= inner (ISTFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType
        | join kind= inner (CRFullSet) 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | where IFP_TaskFinishDate <= ago(1d) and CS_TaskStartDate  > ago(1d) 
          and FS_TaskStartDate  > ago(1d) and IST_TaskStartDate > ago(1d) 
          and CR_TaskStartDate > ago(1d) 
        | summarize by DCShortCode
        ''' 
        };
    
        self.statistics = {
        'T2_IFP_CR_p5080' : 
        '''
        let FY = 
        iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
            ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
        and ProjectType in ("Build","Build PH","OAI Build") 
        and ColoReadyDate <= todatetime(FY_Enddate) 
        and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) 
          by DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate; 
        let IFPFullSet =  DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche
        | join kind = leftouter  
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, Region, ProjectType, 
              IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2 and isnotempty(IFP_TaskStartDate); 
        let IFP_CR_Duration = IFPFullSet 
        | join kind = inner CRFullSet on $left.DCShortCode == $right.DCShortCode 
          and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, IFP_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, IFP_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, IFP_TaskStartDate)*1.0 
        | order by DCShortCode asc 
        | extend ID="IFPCR"
        | where duration  > 0; 
        let IFP_CR_Outlier = IFP_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="IFPCR"; let IFP_CR = IFP_CR_Duration 
        | join kind = leftouter IFP_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = 
        iff(leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier") 
        | where Outlier != "Outlier"; IFP_CR 
        | summarize percentiles(duration, 50,80)
        ''',
        
        'T2_IFP_CR_minmax' : 
        '''
        let FY = iff
        ( 
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
        ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = 
        DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) 
          by DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate; 
        let IFPFullSet =  
        DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche 
        | join kind = leftouter  
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, Region, ProjectType, 
              IFP_TaskStartDate = DesignIFP, IFP_TaskFinishDate = DesignIFP
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2 and isnotempty(IFP_TaskStartDate); 
        let IFP_CR_Duration = IFPFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode 
        and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, IFP_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, IFP_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, IFP_TaskStartDate)*1.0 
        | order by DCShortCode asc | extend ID="IFPCR"
        | where duration  > 0; 
        let IFP_CR_Outlier = IFP_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="IFPCR"; let IFP_CR = IFP_CR_Duration 
        | join kind = leftouter IFP_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = 
          iff(leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier") 
        | where Outlier != "Outlier"; 
        IFP_CR 
        | summarize min(duration), max(duration)
        ''',
        
        'T2_CS_CR_p5080' : 
        '''
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
        ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) 
          by DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate;
        let CSFullSet = DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche
        | join kind = leftouter 
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, Region,
              L2Milestone = "Construction Start", ProjectType, 
              CS_TaskStartDate = ConstructionDate, 
              CS_TaskFinishDate = ConstructionDate
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2 and isnotempty(CS_TaskStartDate)
        | summarize CS_TaskStartDate = minif(CS_TaskStartDate, 
           L2Milestone in ("EarthWorks", "Construction Start")) by 
           DCShortCode, ProjectType, ProjectName, Tranche, Region; 
        let CS_CR_Duration = CSFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, CS_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, CS_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, CS_TaskStartDate)*1.0 
        | order by DCShortCode asc | extend ID="CSCR"| where duration  > 0; 
        let CS_CR_Outlier = CS_CR_Duration 
        | summarize P75 = percentiles(leadtime,75), 
          P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="CSCR"; let CS_CR = CS_CR_Duration 
        | join kind = leftouter CS_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier";CS_CR 
        | summarize percentiles(duration, 50,80)        
        ''',
        
        'T2_CS_CR_minmax' :
        ''' 
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
            ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
           and ProjectType in ("Build","Build PH","OAI Build") 
           and ColoReadyDate <= todatetime(FY_Enddate) 
           and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) by 
           DCShortCode, ProjectType, ProjectName, Tranche, Region, 
           ColoReadyDate;let CSFullSet = DCEPAllMilestonesCurrent
        | where ProjectType in ("Build","Build PH","OAI Build") 
        | summarize by ProjectName, Tranche
        | join kind = leftouter   
        (
            DCEPAllMilestonesCurrent 
            | where Tranche == 1 and ProjectType in ("Build","Build PH","OAI Build") 
            | summarize by DCShortCode, ProjectName,Tranche, Region,
            L2Milestone = "Construction Start", ProjectType, 
            CS_TaskStartDate = ConstructionDate, 
            CS_TaskFinishDate = ConstructionDate
        )
        on $left.ProjectName == $right.ProjectName 
        | project-away ProjectName1, Tranche1
        | where Tranche ==2 and isnotempty(CS_TaskStartDate) 
        | summarize CS_TaskStartDate = minif(
            CS_TaskStartDate, L2Milestone in ("EarthWorks", "Construction Start")
            ) by DCShortCode, ProjectType, ProjectName, Tranche, 
            Region; 
        let CS_CR_Duration = CSFullSet 
        | join kind = inner CRFullSet on $left.DCShortCode == $right.DCShortCode 
          and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, CS_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, CS_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, CS_TaskStartDate)*1.0 
        | order by DCShortCode asc 
        | extend ID="CSCR"
        | where duration  > 0; 
        let CS_CR_Outlier = CS_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),
           P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="CSCR"; let CS_CR = CS_CR_Duration 
        | join kind = leftouter CS_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,
            "Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier";CS_CR 
        | summarize min(duration), max(duration)
        ''',
        
        'T2_FS_CR_p5080' :
        '''
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
            ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) 
        by DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate; 
        let FSFullSet =EstimatedFoundationStart
        | where Tranche ==2
        | summarize by DCShortCode, ProjectType, ProjectName, 
          Tranche,Region,FS_TaskStartDate =Foudation_TaskStartDate; 
        let FS_CR_Duration = FSFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, FS_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, FS_TaskStartDate) 
        | extend leadtime = datetime_diff(
            "day",ColoReadyDate, FS_TaskStartDate)*1.0 
        | order by DCShortCode asc 
        | extend ID="FSCR"
        | where duration  > 0; 
        let FS_CR_Outlier = FS_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="FSCR"; 
        let FS_CR = FS_CR_Duration 
        | join kind = leftouter FS_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier"; 
        FS_CR | summarize percentiles(duration, 50,80)
        ''',
        'T2_FS_CR_minmax' :
        '''
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
            ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) 
          by DCShortCode, ProjectType, ProjectName, Tranche, 
             Region, ColoReadyDate; let FSFullSet =EstimatedFoundationStart
        | where Tranche ==2
        | summarize by DCShortCode, ProjectType, ProjectName, Tranche,Region,
          FS_TaskStartDate =Foudation_TaskStartDate; 
        let FS_CR_Duration = FSFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, FS_TaskStartDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, FS_TaskStartDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, FS_TaskStartDate)*1.0 
        | order by DCShortCode asc 
        | extend ID="FSCR"
        | where duration  > 0; let FS_CR_Outlier = FS_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="FSCR"; let FS_CR = FS_CR_Duration 
        | join kind = leftouter FS_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier"; 
        FS_CR | summarize min(duration), max(duration)
        ''',
        'T2_IST_CR_p5080' :
        '''
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
        ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) by 
          DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate; 
        let ISTFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) by 
          DCShortCode, ProjectType, ProjectName, Tranche, Region, ISTDate; 
        let IST_CR_Duration = ISTFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, ISTDate, ColoReadyDate, 
        duration = datetime_diff("day",ColoReadyDate, ISTDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, ISTDate)*1.0 
        | order by DCShortCode asc | extend ID="ISTCR"
        | where duration  > 0; 
        let IST_CR_Outlier = IST_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="ISTCR"; let IST_CR = IST_CR_Duration 
        | join kind = leftouter IST_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier"; 
        IST_CR | summarize percentiles(duration, 50,80)
        ''',
        'T2_IST_CR_minmax' :
        '''
        let FY = iff(
            monthofyear(ago(0d)) > 6, 
            datetime_part("year", ago(0d)) + 1, 
            datetime_part("year", ago(0d))
        ); 
        let FY_Startdate = strcat(FY - 3, "-07-01"); 
        let FY_Enddate = strcat(FY - 1, "-06-30"); 
        let CRFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) by 
          DCShortCode, ProjectType, ProjectName, Tranche, Region, ColoReadyDate; 
        let ISTFullSet = DCEPAllMilestonesCurrent 
        | where ProjectStatus == "Complete" and Tranche == 2 
          and ProjectType in ("Build","Build PH","OAI Build") 
          and ColoReadyDate <= todatetime(FY_Enddate) 
          and ColoReadyDate > todatetime(FY_Startdate) 
        | summarize minif(ColoReadyDate, Tranche == 2) by 
          DCShortCode, ProjectType, ProjectName, Tranche, Region, ISTDate; 
        let IST_CR_Duration = ISTFullSet 
        | join kind = inner CRFullSet 
        on $left.DCShortCode == $right.DCShortCode and $left.ProjectType == $right.ProjectType 
        | project DCShortCode, ISTDate, ColoReadyDate, 
          duration = datetime_diff("day",ColoReadyDate, ISTDate) 
        | extend leadtime = datetime_diff("day",ColoReadyDate, ISTDate)*1.0 
        | order by DCShortCode asc | extend ID="ISTCR"
        | where duration  > 0; 
        let IST_CR_Outlier = IST_CR_Duration 
        | summarize P75 = percentiles(leadtime,75),P25 = percentiles(leadtime,25) 
        | extend IQR = P75-P25 
        | extend Upper_Whisker = P75+1.5*IQR ,Lower_Whisker = P25-1.5*IQR 
        | extend ID="ISTCR"; 
        let IST_CR = IST_CR_Duration 
        | join kind = leftouter IST_CR_Outlier on $left.ID == $right.ID 
        | extend Outlier = iff(
            leadtime >Upper_Whisker or leadtime < Lower_Whisker,"Outlier","Not Outlier"
            ) 
        | where Outlier != "Outlier"; 
        IST_CR | summarize min(duration), max(duration)
        '''
        };
     
    def query_dc_list(self):
        self.data = {};
        query = self.dcshortcode_selection['T2_IST2CR_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['T2_IST2CR_List'] = result;

        query = self.dcshortcode_selection['T2_FS2CR_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['T2_FS2CR_List'] = result;

        query = self.dcshortcode_selection['T2_CS2CR_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['T2_CS2CR_List'] = result;

        query = self.dcshortcode_selection['T2_IFP2CR_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['T2_IFP2CR_List'] = result;
        
        query = self.dcshortcode_selection['T2_DC_List'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
        self.data['T2_DC_List'] = result;
        
        query = self.statistics['T2_IFP_CR_p5080'];
        response = self.kusto_client.query(query);
        result = response.reset_index();
    
        self.data['T2_IFP_CR_p5080'] = result;    

        self.kusto_client.close();

        return self.data;


a = DCShortCodeSelection();
data = a.query_dc_list()
N = 10000
stages = 1

projects = data['T2_DC_List'];
print(projects);
ist2cr_project_list = data['T2_IST2CR_List'];
print(ist2cr_project_list);
fs2cr_project_list = data['T2_FS2CR_List'];
print(fs2cr_project_list);
cs2cr_project_list = data['T2_CS2CR_List'];  
print(cs2cr_project_list);
ifp2cr_project_list = data['T2_IFP2CR_List'];
print(ifp2cr_project_list);

count = 0;
max_loop = projects.size;



     index DCShortCode
0        0       AKL02
1        1       AMS13
2        2       AMS14
3        3       ATL06
4        4       AVC01
..     ...         ...
121    121       YQB04
122    122       YQB06
123    123       YTO11
124    124       YTO14
125    125       YTO40

[126 rows x 2 columns]
   index DCShortCode
0      0       IAD11
1      1       IAD03
2      2       DSM14
3      3       DSM40
    index DCShortCode
0       0       AMS13
1       1       AMS14
2       2       SYD08
3       3       CHI05
4       4       PHX71
5       5       DUB15
6       6       MAD02
7       7       SAT14
8       8       SAT15
9       9       WAW03
10     10       AKL02
11     11       EAT03
12     12       LVL04
13     13       LVL06
14     14       LVL09
15     15       LVL10
16     16       SYD05
17     17       DSM12
18     18       DSM13
19     19       CYS17
20     20       CYS13
21     21       LON01
22     22       GVX02
23     23       JKT05
24     24       CPH01
25     25         CO9
2